In [27]:
# by caleb fried 04/2022

#initilization
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import savemat
import os
import h5py as h5
import time
ax = {'x':1,'y':0,'z':2}
#define a dict of axes to make things simpler
#x y z labeling based on labels on Attocube controller

In [28]:
from auspex.instruments import SR865
from pyanc350.v4 import Positioner

In [71]:
lockin = SR865()
lockin.connect('GPIB0::9::INSTR')   
anc = Positioner()
#instantiate positioner as anc
print('-------------------------------------------------------------')
print('capacitances:')
for axis in sorted(ax.keys()):
    print(axis,anc.measureCapacitance(ax[axis]))
    anc.setFrequency(ax[axis],100)
    anc.setAmplitude(ax[axis],15)
anc.setAmplitude(2,30)

auspex-WARNING: 2022-05-23 14:42:31,902 ----> User-provided IP {} is a valid IP address but does not appear to be in human-readable format.
-------------------------------------------------------------
capacitances:
x 1.80642e-06
y 1.789258e-06
z 1.256061e-06


In [19]:
lockin.x

0.0056804935448

In [72]:
Xtarget = 11.611e-3-1.688e-3  # in m

anc.setAxisOutput(ax['x'], 1, 0)
anc.setTargetRange(ax['x'],1e-6)
anc.setTargetPosition(ax['x'], Xtarget)
anc.startAutoMove(ax['x'], 1, 0)

In [33]:
Ytarget = 8.52e-3 # in m
if Ytarget < 6e-3:
    error('lens will crash into sample!')

anc.setAxisOutput(ax['y'], 1, 0)
anc.setTargetRange(ax['y'],1e-6)
anc.setTargetPosition(ax['y'], Ytarget)
anc.startAutoMove(ax['y'], 1, 0)

In [68]:
Ztarget = 5e-3 # in m

anc.setAxisOutput(ax['z'], 1, 0)
anc.setTargetRange(ax['z'],1e-6)
anc.setTargetPosition(ax['z'], Ztarget)
anc.startAutoMove(ax['z'],1, 0)

In [69]:
# assume we start above the Si void
step1 = 0
step2 = 0
step3 = 0
i = 0
data = [0] * 100000
print('starting scan at',anc.getPosition(ax['z']))
anc.startContinuousMove(ax['z'],1,1) # assuming right is forward
while step1 == 0:
    i = i+1
    time.sleep(0.01)
    data[i] = lockin.x
    if i>2 and (data[i] - data[i-2])/data[i-2] < -0.1:
        step1 = 1
anc.startContinuousMove(ax['z'],0,1)

print('accounting for rotations...')
start = anc.getPosition(ax['z'])                         
anc.setAxisOutput(ax['z'], 1, 1)
anc.setTargetRange(ax['z'],1e-6)
anc.setTargetPosition(ax['z'], start - 0.050e-3) # assuming right is forward
anc.startAutoMove(ax['z'], 1, 0)

print('reached Si at',anc.getPosition(ax['z']))
input("Press Enter to continue...")

print('-------------------------------------------------------------')
i = 0
data = [0] * 100000
anc.startContinuousMove(ax['x'],1,0)
while step2 == 0:
    i = i+1
    time.sleep(0.01)
    data[i] = lockin.x
    if i>2 and (data[i] - data[i-2])/data[i-2] > 0.1:
        step2 = 1
anc.startContinuousMove(ax['x'],0,0)
print('reached wire bond at',anc.getPosition(ax['x']))
input("Press Enter to continue...")

print('-------------------------------------------------------------')
i = 0
data = [0] * 100000
anc.startContinuousMove(ax['x'],1,0)
time.sleep(3)
while step3 == 0:
    i = i+1
    time.sleep(0.01)
    data[i] = lockin.x
    if i>2 and (data[i] - data[i-2])/data[i-2] > 0.1:
        step3 = 1
anc.startContinuousMove(ax['x'],0,0)
print('reached corner at',anc.getPosition(ax['x']))
input("Press Enter to continue...")

print('-------------------------------------------------------------')
print('moving to CPW')
start = anc.getPosition(ax['z'])                         
anc.setAxisOutput(ax['z'], 1, 1)
anc.setTargetRange(ax['z'],1e-6)
anc.setTargetPosition(ax['z'], start - 2.00863e-3) # assuming right is forward
anc.startAutoMove(ax['z'], 1, 0)

#check what's happening
time.sleep(0.5)
moving = 1
target = 0
while target == 0:
    connected, enabled, moving, target, eotFwd, eotBwd, error = anc.getAxisStatus(ax['z']) #find bitmask of status
    if target == 0:
        print('axis moving, currently at',anc.getPosition(ax['z']))
    elif target == 1:
        print('axis arrived at',anc.getPosition(ax['z']))
        anc.startAutoMove(ax['z'], 0, 0)
    time.sleep(0.5)
input("Press Enter to continue...")

print('-------------------------------------------------------------')
print('moving to sample area')
start = anc.getPosition(ax['x'])                         
anc.setAxisOutput(ax['x'], 1, 0)
anc.setTargetRange(ax['x'],1e-6)
anc.setTargetPosition(ax['x'], start + 0.96e-3) #assuming positive x is right
anc.startAutoMove(ax['x'], 1, 0)

#check what's happening
time.sleep(0.5)
moving = 1
target = 0
while target == 0:
    connected, enabled, moving, target, eotFwd, eotBwd, error = anc.getAxisStatus(ax['x']) #find bitmask of status
    if target == 0:
        print('axis moving, currently at',anc.getPosition(ax['x']))
    elif target == 1:
        print('axis arrived at',anc.getPosition(ax['x']))
        anc.startAutoMove(ax['x'], 0, 0)
    time.sleep(0.5)
print('end position: x =',anc.getPosition(ax['x']),', z =',anc.getPosition(ax['z']))

starting scan at 0.004999932
accounting for rotations...
reached Si at 0.004747252
Press Enter to continue...
-------------------------------------------------------------
reached wire bond at 0.00860291
Press Enter to continue...
-------------------------------------------------------------
reached corner at 0.00893153
Press Enter to continue...
-------------------------------------------------------------
moving to CPW
axis moving, currently at 0.004673944
axis moving, currently at 0.004655294
axis moving, currently at 0.004627393
axis moving, currently at 0.004606043
axis moving, currently at 0.00458695
axis moving, currently at 0.004574296
axis moving, currently at 0.004549925
axis moving, currently at 0.004528278
axis moving, currently at 0.00451144
axis moving, currently at 0.00448678
axis moving, currently at 0.004458298
axis moving, currently at 0.004443593
axis moving, currently at 0.004423056
axis moving, currently at 0.004399114
axis moving, currently at 0.004377635
axis mov

axis moving, currently at 0.011267033
axis moving, currently at 0.011277317
axis moving, currently at 0.011309949
axis moving, currently at 0.011333684
axis moving, currently at 0.011361171
axis moving, currently at 0.011386217
axis moving, currently at 0.011402801
axis moving, currently at 0.011417562
axis moving, currently at 0.011447569
axis moving, currently at 0.011470008
axis moving, currently at 0.011501804
axis moving, currently at 0.01152356
axis moving, currently at 0.011548801
axis moving, currently at 0.0115642
axis moving, currently at 0.011595549
axis arrived at 0.011612627
end position: x = 0.011610433 , z = 0.002688889


In [73]:
#cleanup

anc.disconnect()
lockin.disconnect()